# Azure OpenAI Function Calling을 다음 검색 및 카카오 모빌리티 API와 통합 예시
이 파일은 다음 웹 검색과 Kakao Mobility의 실시간 검색 정보를 활용할 수 있는 GPT 모델 사용 예시입니다.

참고: 이 예시는 외부 API를 호출하도록 만들어져 있습니다. 따라서 [KAKAO API](https://developers.kakao.com/console/app)에서 제공하는 `KEY를 발급받아 .env 파일에 등록`하고 사용하여야 합니다.

## Setup

In [1]:
import openai
import json
import os
import requests
import pytz
from urllib import parse
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

# Azure OpenAI resource 정보를 설정합니다.
openai.api_type     = os.getenv("OPENAI_API_TYPE")
openai.api_key      = os.getenv("OPENAI_API_KEY")
openai.api_base     = os.getenv("OPENAI_API_BASE")
openai.api_version  = os.getenv("OPENAI_API_VERSION")   # API 버전은 "2023-07-01-preview" 부터 사용 가능합니다.
deployment_id       = os.getenv("DEPLOYMENT_NAME_16K")  # Azure OpenAI resource의 deployment id를 입력합니다.
KAKAO_API_KEY       = os.getenv("KAKAO_REST_API_KEY")   # KAKAO REST API 키입니다.

## 1. Kakao API functions 정의
이제 함수로 작업하는 방법을 알았으므로 코드에서 몇 가지 함수를 정의하여 함수를 사용하는 프로세스를 끝까지 살펴보겠습니다.

In [2]:
headers = {
    "Authorization": "KakaoAK " + KAKAO_API_KEY,
    "Content-Type": "application/json",
}    

# Kakao 키워드 기반 위경도 좌표 찾기
def get_location_xy(keyword="판교 알파돔타워"):
    params = {
        "query": keyword
    }
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?" + parse.urlencode(params)
    response = requests.get(url, headers=headers)
    return (response.json()["documents"][0]["x"] + "," + response.json()["documents"][0]["y"] + ",name=" + response.json()["documents"][0]["place_name"])

def get_current_time():
    try:
        # Get the timezone for the city
        timezone = pytz.timezone("Asia/Seoul")

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y%m%d%H%M")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."    

### `Function #1`: 카카오 모빌리티 길찾기 API

카카오 길찾기 API를 통해서 출발지와 목적지 사이의 정보를 탐색합니다.

In [3]:
# Kakao 길찾기 API
def get_directions(origin, destination, waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    origin_xy_info = get_location_xy(origin)
    destin_xy_info = get_location_xy(destination)
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
    }
    url = "https://apis-navi.kakaomobility.com/v1/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": response_summary["distance"],
        "duration": response_summary["duration"],
    }
    
    return json.dumps(return_data)


### `Function #2`: 카카오 모빌리티 미래 길찾기 API

지정된 날짜와 시간으로 카카오 길찾기 API를 통해서 출발지와 목적지 사이의 정보를 탐색합니다.

In [4]:
# Kakao 미래 길찾기 API
def get_future_directions(origin, destination, departure_time=get_current_time(), waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    origin_xy_info = get_location_xy(origin)
    destin_xy_info = get_location_xy(destination)
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
        "departure_time": departure_time,
    }
    url = "https://apis-navi.kakaomobility.com/v1/future/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": response_summary["distance"],
        "duration": response_summary["duration"],
    }
    
    return json.dumps(return_data)


### `Function #3`: 실시간 지역시간 수집

지역별 실시간 시간 확인

In [5]:
def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y%m%d%H%M")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."

### `Function #4`: 다음 검색 API 연결

Daum 검색 API는 포털 사이트 Daum에서 방대한 웹 문서, 동영상, 이미지, 블로그, 책, 카페를 검색하는 기능을 제공합니다. 검색 결과는 JSON 객체로 전달돼 서비스에서 자유롭게 출력하거나 활용할 수 있습니다.

참고: https://developers.kakao.com/docs/latest/ko/daum-search/common

In [34]:
# 검색어를 기반으로 웹에서 검색 결과를 요약해 주는 함수
def daum_web_search(query_msg, sort="accuracy", page=1, size=3):

    params = {
        "query": query_msg,
        "sort": sort,
        "page": page,
        "size": size,
    }
    url = "https://dapi.kakao.com/v2/search/web?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    # print(response)
    return_data = response.json()["documents"]
    print(return_data)

    
    return json.dumps(return_data)

## 2. GPT를 사용한 `Function` 호출

`Function Calling`을 위한 단계: 

1. 사용자 쿼리와 functions 매개변수(parameter)에 정의된 함수 집합을 사용하여 모델을 호출합니다.
2. 모델은 함수 호출을 선택할 수 있습니다. 콘텐츠는 사용자 지정 스키마를 준수하는 문자열화된 JSON 객체가 됩니다(참고: 모델이 잘못된 JSON 또는 환각(hallucination) 매개변수를 생성할 수 있음).
3. 코드에서 문자열을 JSON으로 구문 분석합니다. 제공된 인수가 있는 경우 해당 인수로 함수를 호출합니다.
4. 함수 응답을 새 메시지로 추가하여 모델을 다시 호출하고 모델이 결과를 사용자에게 다시 요약하도록 합니다.

### 2.1 모델이 호출 방법을 알 수 있도록 함수 설명

In [35]:
# caculate from 초 to 시 분 초
def time_calculator(seconds):
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    
    if hours == 0:
        return "%02d분 %02d초" % (minutes, seconds)
    elif minutes == 0:
        return "%02d초" % (seconds)
    else:
        return "%02d시간 %02d분 %02d초" % (hours, minutes, seconds)

In [36]:

functions = [
        {
            "name": "get_directions",
            "description": "API to search routes based on origin and destination information",
            # "description": "출발지와 도착지 정보를 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},                    
                },
                "required": ["origin", "destination"],
            },
        },
        {
            "name": "get_future_directions",
            "description": "API to search routes based on origin and destination information based on future departure_time",
            # "description": "출발지와 도착지 정보를 미래 시간 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "departure_time": {"type": "string"},
                },
                "required": ["origin", "destination", "departure_time"],
            },
        },
        {
            "name": "get_current_time",
            "description": "Convert seconds to hours minutes seconds",
            # "description": "초를 시분초 단위로 변환",
            "parameters": {
                "type": "object",
                "properties": {
                    "duration": {"type": "integer"},
                },
                "required": ["duration"],
            },
        },
        {
            "name": "daum_web_search",
            "description": "A function that summarizes web search results for the query",
            # "description": "검색어를 기반으로 웹에서 검색 결과를 요약해 주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_msg": {"type": "string"},
                    "size": {"type": "integer",
                        "description": "Number of web search results",
                    },
                },
                "required": ["query_msg"],
            },
        },
    ]

available_functions = {
            "get_directions": get_directions,
            "get_future_directions": get_future_directions,
            "get_current_time": get_current_time,
            "daum_web_search": daum_web_search
        } 

### 2.2 function call을 검증하는 helper 함수 정의
모델이 잘못된 function call을 생성할 수 있으므로 function의 유효성을 검사하는 것이 중요합니다. 여기서는 사용 사례에 대해 더 복잡한 유효성 검사를 적용할 수 있지만 함수 호출의 유효성을 검사하는 간단한 helper 함수를 정의합니다.

In [37]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [38]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        # adding assistant response to messages
        
        # function_name 값에 따른 분기 처리
        if function_name == "get_directions" or function_name == "get_future_directions":
            messages.append(
                {"role": "system", "content": "You are a bot that guides you through car routes. When the user provides the origin and destination name, you provides summary route guidance information.",}
            )
        elif function_name == "get_current_time":
            messages.append(
                {"role": "system", "content": "You are a bot that tells the world time. You describe based on the given data and do not judge and create other sentences."},
            )
        elif function_name == "daum_web_search":
            messages.append(
                {"role": "system", "content": "You are an agent that summarizes web search results for a query. You need to summarize based on JSON data retrieved from a web search. You only speak Korean."},
            )
        else :
            messages.append(
                {"role": "system", "content": "You are an AI assistant that helps people find information. The answer must be judged and answered based on factual data. Please use simple expressions as much as possible."},
            )

        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        # print("Messages in second request:")
        # for message in messages:
        #     print(message)
        # print()
        # print(json.dumps(messages, ensure_ascii=False, indent=4))
        # print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

## 3. 다음 검색 결과 요약

Bing Chat과 같은 서비스를 Daum 검색 엔진 기반으로 만들어 볼 수 있습니다.

In [40]:
prompt_msg = "손흥민 리버풀전 평점과 활약상"

messages = [
    {"role": "system", "content": "You are an agent that summarizes web search results for a query. You need to summarize based on JSON data retrieved from a web search. You only speak Korean."},
    {"role": "user", "content": f"Summarize web search results for '{prompt_msg}'"}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\\\"", "\"")
print(content)

[{'contents': '0000071452 &#39;<b>손흥민</b>이 <b>리버풀</b><b>전</b> 팀 내 최고 점수를 받았다. 최저 점은 에릭 다이어였다. 경기 후 영국 매체...최고로 꼽았다. &#39;득점 장면에 대한 설명과 함께 <b>평점</b> 8점을 부여했다. 선제 골의 주인공 케인과 훌륭...', 'datetime': '2023-05-01T03:11:00.000+09:00', 'title': '[정보] SON, <b>리버풀</b><b>전</b> 최고<b>평점</b> 팀내! 최저는 다이어..&#39;1골 1도움 <b>손흥민</b>...&#39;', 'url': 'http://www.gasengi.com/main/board.php?bo_table=football04&wr_id=841504'}, {'contents': '자리 잡는 모습을 보여줬다. 자세한 내용은 <b>손흥민</b>/2017-18 시즌 문서 참고하십시오. 2017-18 시즌: 53경기...골 가뭄을 계속 이어가면서 10월 말 9R <b>리버풀</b> <b>전</b>에서야 리그 첫 골을 기록하였다. 그리고 10월...', 'datetime': '2023-11-14T00:00:00.000+09:00', 'title': '<b>손흥민</b>/토트넘 홋스퍼 FC - 나무위키', 'url': 'https://namu.wiki/w/%EC%86%90%ED%9D%A5%EB%AF%BC/%ED%86%A0%ED%8A%B8%EB%84%98%20%ED%99%8B%EC%8A%A4%ED%8D%BC%20FC'}, {'contents': '토트넘, <b>리버풀</b>, 도르트문트 등 해외 다양한 클럽들과 꾸준히 링크가 돌던 <b>손흥민</b>은 결국 후보를...2014 브라질 월드컵 최종예선 마지막 경기인 이란<b>전</b>을 앞두고 레버쿠젠으로의 공식 이적을 확정지었다. 그리고...', 'datetime': '2023-10-18T00:00:00.000+09:00', 'title': '<b>손흥민</b>/바

## 4. 경로 검색 요청

In [ ]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are a navigation bot agent. Your reply must absolutely in JSON format. You must never modify or tamper with data."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 JSON 포맷으로 알려줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\\\"", "\"")
print(content)

"{
  "route_info": {
    "origin": "판교 알파돔타워",
    "destination": "한국마이크로소프트",
    "taxi_fare": "32600",
    "toll_fare": "0",
    "distance": "29264",
    "estimated_time": "3590"
  },
  "status": "success"
}"


In [13]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are a bot that guides you through car routes. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. \
    If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 목록을 만들어서 알려줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\"", "")
print(content)

Recommended Function call:
{
  "name": "get_directions",
  "arguments": "{\n  \"origin\": \"\ud310\uad50 \uc54c\ud30c\ub3d4\ud0c0\uc6cc\",\n  \"destination\": \"\ud55c\uad6d\ub9c8\uc774\ud06c\ub85c\uc18c\ud504\ud2b8\"\n}"
}

Output of function call:
{"origin_name": "\uc54c\ud30c\ub3d4\ud0c0\uc6cc", "destination_name": "\ud55c\uad6d\ub9c8\uc774\ud06c\ub85c\uc18c\ud504\ud2b8", "taxi_fare": 33200, "toll_fare": 0, "distance": 30056, "duration": 3650}
Messages in second request:
{'role': 'system', 'content': 'You are a bot that guides you through car routes. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily.     If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should b

In [12]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are car route navigation. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. Summalize sentance without list. \
    If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 문장으로 요약해줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\"", "")
print(content)

판교 알파돔타워에서 한국마이크로소프트까지는 약 30.6km 이동해야 하며, 예상 소요 시간은 약 35분 24초입니다. 택시 요금은 33,500원이며 톨비는 없습니다.


## 5. 미래 운행 정보 기반의 검색

미래 운행 정보 기반의 검색 API를 자동 호출하도록 출발시간을 Prompt에 제공합니다.

In [11]:
origin_name = "한국마이크로소프트"
destin_name = "판교 알파돔타워"

messages = [
        {"role": "system", "content": "You are car route navigation. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. Summalize sentance without list. \
        If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
        {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 출발시간이 2023년 12월 25일 17시 0분(%Y%m%d%H%M) 일때, 경로 검색한 정보를 문장으로 요약해줘."}
    ]
assistant_response1 = run_conversation(messages, functions, available_functions, deployment_id)
future_content = json.dumps(assistant_response1['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
future_content = future_content.replace("\\n", "\n").replace("\"", "")
print(future_content)

출발지인 한국마이크로소프트에서 목적지인 판교 알파돔타워까지의 길을 안내해 드리겠습니다. 차량으로 이동 시 예상 거리는 약 25.19km이며, 소요 시간은 대략 1시간 6분 17초 입니다. 택시 요금은 대략 31,400원이며, 추가로 통행료 1,000원이 발생할 예정입니다. 단, 실제 교통 상황에 따라 시간과 요금이 다소 변동될 수 있습니다. 2023년 12월 25일 17시에 출발하시면 됩니다.
